<a href="https://colab.research.google.com/github/juancarloscaraguay1-lang/Ejercicios-KNN/blob/main/Copia_de_Copia_de_Ejercicios_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/akshayrb22/playing-with-data/blob/master/supervised_learning/KNN/KNN.ipynb)

El algoritmo de K vecinos más cercanos (KNN) es un tipo de algoritmo de aprendizaje automático supervisado.

KNN es extremadamente fácil de implementar en su forma más básica, pero realiza tareas de clasificación bastante complejas.

Es un algoritmo de aprendizaje perezoso, ya que no tiene una fase de entrenamiento especializada.

En su lugar, utiliza todos los datos para el entrenamiento mientras clasifica un nuevo punto o instancia de datos.

KNN es un **algoritmo de aprendizaje no paramétrico, lo que significa que no asume nada sobre los datos subyacentes**.

Esta característica es extremadamente útil, ya que la mayoría de los datos del mundo real no siguen ningún supuesto teórico, como la separabilidad lineal, la distribución uniforme, etc.

## Ventajas

- Es extremadamente fácil de implementar.

- Como se mencionó anteriormente, es un algoritmo de aprendizaje perezoso y, por lo tanto, no requiere entrenamiento previo para realizar predicciones en tiempo real. Esto hace que el algoritmo KNN sea mucho más rápido que otros algoritmos que requieren entrenamiento, como SVM, regresión lineal, etc.

- Dado que el algoritmo no requiere entrenamiento previo para realizar predicciones, se pueden agregar nuevos datos sin problemas.

- Solo se requieren dos parámetros para implementar KNN: el valor de K y la función de distancia (p. ej., euclidiana o Manhattan, etc.).

## Desventajas

- El algoritmo KNN no funciona bien con datos de alta dimensión, ya que, con un gran número de dimensiones, le resulta difícil calcular la distancia en cada dimensión.

- El algoritmo KNN tiene un alto coste de predicción para conjuntos de datos grandes. Esto se debe a que, en conjuntos de datos grandes, el coste de calcular la distancia entre un nuevo punto y cada punto existente es mayor.

- Finalmente, el algoritmo KNN no funciona bien con características categóricas, ya que es difícil calcular la distancia entre dimensiones con características categóricas.

In [31]:
#Importando las librerías necesarias
import pandas as pd
import numpy as np
import math
import operator
print(operator)

<module 'operator' from '/usr/local/python/3.12.1/lib/python3.12/operator.py'>


# Importando el Dataset

In [32]:
url = 'https://raw.githubusercontent.com/melwinlobo18/K-Nearest-Neighbors/master/Dataset/data.csv'
df = pd.read_csv(url)  # Dataset - Datos de cáncer de mama de Wisconsin
df['diagnosis'] = df['diagnosis'].map({
    'M': 1,
    'B': 2
})  # Valores de las etiquetas - 1 para Maligno y 2 para Benigno
labels = df['diagnosis'].tolist()
df['Class'] = labels  #Copiando los valores del diagnóstico a una nueva columna de etiquetas
df = df.drop(['id', 'Unnamed: 32', 'diagnosis'],
             axis=1)  #Borramos las columnas innecesarias
df.head()  #Imprimimos las 5 primeras filas del Dataset

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


# Partición de entrenamiento y de test

In [33]:
np.random.seed(1)
msk = np.random.rand(
    len(df)) < 0.7  #Un arreglo (vector) que contine True (con probabilidad 0.7) y False
train = df[msk]  #Filas que contienen valores True del arreglo
test = df[~msk]  #Filas que contienen valores False del arreglo
print('Número de observaciones el los datos de entrenamiento: ', len(train))
print('Número de observaciones en los datos de testing: ', len(test))

Número de observaciones el los datos de entrenamiento:  395
Número de observaciones en los datos de testing:  174


In [34]:
train.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Class
0,17.99,10.38,122.80,1001.0,0.1184,0.2776,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
2,19.69,21.25,130.00,1203.0,0.1096,0.1599,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.1425,0.2839,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.1003,0.1328,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1
5,12.45,15.70,82.57,477.1,0.1278,0.1700,0.1578,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,1


In [35]:
test.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Class
1,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.24160,0.18600,0.2750,0.08902,1
13,15.850,23.95,103.70,782.7,0.08401,0.10020,0.09938,0.05364,0.1847,0.05338,...,27.66,112.00,876.5,0.1131,0.1924,0.23220,0.11190,0.2809,0.06287,1
20,13.080,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.03110,0.1967,0.06811,...,20.49,96.09,630.5,0.1312,0.2776,0.18900,0.07283,0.3184,0.08183,2
21,9.504,12.44,60.34,273.9,0.10240,0.06492,0.02956,0.02076,0.1815,0.06905,...,15.66,65.13,314.9,0.1324,0.1148,0.08867,0.06227,0.2450,0.07773,2
24,16.650,21.38,110.00,904.6,0.11210,0.14570,0.15250,0.09170,0.1995,0.06330,...,31.56,177.00,2215.0,0.1805,0.3578,0.46950,0.20950,0.3613,0.09564,1


# Distancia Euclidiana

Se utiliza para medir la distancia entre las muestras p y q en un espacio de características de n dimensiones:

<img src="https://drive.google.com/uc?export=view&id=13p3YOEqHybBrN8P5wpsPwBC6-7v3N98f">

Por ejemplo, imagínelo como una línea "recta que conecta" en un espacio de características 2D:

<img src="https://drive.google.com/uc?export=view&id=1Qc0nBRJTWBrJeKZvvj8dVk7YZJrvMpn8">

In [36]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)
    print(distance)



# Calculando los vecinos más cercanos

Supongamos que nuestros datos están dispersos como se muestra a continuación. Los círculos rojos representan la malignidad y los verdes, el benignidad.

<img src="https://drive.google.com/uc?export=view&id=1oEbIjxE0Ysdo5fEeNU5CGo4ruP544Ywz">

En este ejemplo estamos calculando k = 3 vecinos más cercanos, por lo que la curva de ajuste se verá así:

<img src="https://drive.google.com/uc?export=view&id=1amCw22CqF0ideK9VWNIWQAASpZOWGPwi">

In [37]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []  #Lista que guarda todos los valores de distancias
    length = len(testInstance) - 1  #Largo de la lista
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)  #Calculando las distancias euclídeas
        distances.append((trainingSet[x],dist))  #Añadimos los valores de la distancia al final de la lista
        print((trainingSet[x],dist))
    distances.sort(key=operator.itemgetter(1))  #Ordenamos basados en los valores de la distancia
    neighbors = []  #Lista para guardar a todos los vecinos

    for x in range(k):
        neighbors.append(distances[x][0])  #El número de vecinos depende del valor de K
        return neighbors

In [38]:
def getResponse(neighbors):
    classVotes = {}  #Diccionario para guardar las etiquetas y su conteo
    for x in range(len(neighbors)):
        response = neighbors[x][-1]  #Valores de las etiquetas de los vecinos
        print
        if response in classVotes:
            classVotes[
                response] += 1  #Si el valor de la etiqueta ya esta presente incrementamos 1
        else:
            classVotes[
                response] = 1  #Si el valor de la etiqueta no esta presente añadimos la etiqueta al diccionario
    sortedVotes = sorted(
        classVotes.items(), key=operator.itemgetter(1), reverse=True
    )  #Ordenamos el diccionario basados en el valor de conteo en orden descendente
    return sortedVotes[0][
        0]  #Retornamos la etiqueta con el número más alto de ocurrencias

# Calculando la exactitud

La exactitud es una métrica que informalmente se refiere a la fracción de predicciones correctas de nuestro modelo. Formalmente, la precisión se define de la siguiente manera:

<img src="https://drive.google.com/uc?export=view&id=1gk-T6kanOB6mM_bs54jgyMs0Bu93V4sh">

In [39]:
from re import X
def getAccuracy(testSet, predictions):
    correct = 0  #Variable para guardar las predicciones correctas
    for x in range(len(testSet)):
        if testSet[x][-1] is predictions[x]:  #Validamos que los valores predichos sean los mismos que el valor de la etiqueta
            correct += 1  #Incrementado cuando ambos valores son los mismos

    return (correct / float(len(testSet))) * 100.0  #Exactitud = Número de predicciones correctas / Número total del predicciones
    print(correct)

# Implementación en un dataset mas pequeño

In [40]:
trainSet = [[5, 1, 1, 1, 2, 1, 3, 2, 1, 2],
            [10, 10, 10, 10, 5, 10, 10, 10, 7, 4]]
testInstance = [4, 8, 6, 4, 3, 4, 10, 6, 1, 2]
k = 1
neighbors = getNeighbors(trainSet, testInstance, k)
print(neighbors)

([5, 1, 1, 1, 2, 1, 3, 2, 1, 2], 12.609520212918492)
([10, 10, 10, 10, 5, 10, 10, 10, 7, 4], 13.564659966250536)
[[5, 1, 1, 1, 2, 1, 3, 2, 1, 2]]


Es la lista de vecinos más cercanos,
Probablemente en tu KNN, cada vecino es un vector de características más su etiqueta final en el último elemento.

In [41]:
neighbors = [[5, 1, 1, 1, 2, 1, 3, 2, 1, 2], [3, 1, 1, 1, 2, 1, 2, 3, 1, 2],
             [10, 10, 10, 10, 5, 10, 10, 10, 7, 4]]
response = getResponse(neighbors)
print(response)

2


In [42]:
testSet = [[5, 1, 1, 1, 2, 1, 3, 2, 1, 2], [3, 1, 1, 1, 2, 1, 2, 3, 1, 2],
           [10, 10, 10, 10, 5, 10, 10, 10, 7, 4]]
predictions = [2, 2, 2]
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


# Implementación en el dataset importado

Este codigo recorre cada fila del set de prueba, luego busca los 3 vecinos mas cercanos, luego obtierne la prediccion basada en vecinos y guarda y muestra la predicción junto al valor real

In [43]:
predictions = []  #Lista para guardar los valores predichos
k = 3  # 3-Nearest Neighbors
trainingSet = train.values.tolist()  #Lista conteniendo los datos de entrenamiento
testSet = test.values.tolist()  #Lista conteniendo los datos de testing
for x in range(len(testSet)):
    neighbors = getNeighbors(trainingSet, testSet[x], k)
    result = getResponse(neighbors)
    predictions.append(result)  # Guardando los valores predichos.
    print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))

([17.99, 10.38, 122.8, 1001.0, 0.1184, 0.2776, 0.3001, 0.1471, 0.2419, 0.07871, 1.095, 0.9053, 8.589, 153.4, 0.006399, 0.04904, 0.05373, 0.01587, 0.03003, 0.006193, 25.38, 17.33, 184.6, 2019.0, 0.1622, 0.6656, 0.7119, 0.2654, 0.4601, 0.1189, 1.0], 341.7302620944424)
([19.69, 21.25, 130.0, 1203.0, 0.1096, 0.1599, 0.1974, 0.1279, 0.2069, 0.05999, 0.7456, 0.7869, 4.585, 94.03, 0.00615, 0.04006, 0.03832, 0.02058, 0.0225, 0.004571, 23.57, 25.53, 152.5, 1709.0, 0.1444, 0.4245, 0.4504, 0.243, 0.3613, 0.08758, 1.0], 276.7762394417638)
([11.42, 20.38, 77.58, 386.1, 0.1425, 0.2839, 0.2414, 0.1052, 0.2597, 0.09744, 0.4956, 1.156, 3.445, 27.23, 0.00911, 0.07458, 0.05661, 0.01867, 0.05963, 0.009208, 14.91, 26.5, 98.87, 567.7, 0.2098, 0.8663, 0.6869, 0.2575, 0.6638, 0.173, 1.0], 1679.2374812295761)
([20.29, 14.34, 135.1, 1297.0, 0.1003, 0.1328, 0.198, 0.1043, 0.1809, 0.05883, 0.7572, 0.7813, 5.438, 94.44, 0.01149, 0.02461, 0.05688, 0.01885, 0.01756, 0.005115, 22.54, 16.67, 152.2, 1575.0, 0.1374, 0.2

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



# Matriz de Confusión

Mide el rendimiento para problemas de clasificación de aprendizaje automático, donde el resultado puede ser de dos o más clases. Es una tabla con cuatro combinaciones diferentes de valores predichos y reales.

<img src="https://drive.google.com/uc?export=view&id=1hYthqmYW82QHRFnIpiGQDLt4r44aurv7">

Este código:

Extrae las etiquetas reales del conjunto de prueba (y_test).
Compara estas etiquetas con las predicciones (predictions).
Calcula e imprime la matriz de confusión, que muestra cuántas veces el modelo acertó o falló en cada clase.

In [44]:
print("Confusion Matrix")
y_test = []
for i in testSet:
    y_test.append(i[30])
from sklearn.metrics import confusion_matrix, accuracy_score
res = confusion_matrix(y_test, predictions)
print("Valores reales:", y_test)

Confusion Matrix
Valores reales: [1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0]


# Exactitud

In [45]:
accuracy = accuracy_score(y_test, predictions) * 100
print('Accuracy: ' + repr(accuracy) + '%')

Accuracy: 91.95402298850574%
